<a href="https://colab.research.google.com/github/Shadow-Walker360/Shadow-Walker360/blob/main/aviator_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer
import json
import re
import logging
from logging.handlers import RotatingFileHandler
import matplotlib.pyplot as plt

# Constants
BASE_URL = 'https://www.betika.com/en-ke/aviator'
ROUNDS = 100
THRESHOLD = 1.5
AUTH_TOKEN = 'your_auth_token'
BET_AMOUNT = 100

# Set up logging
logger = logging.getLogger('aviator_game')
logger.setLevel(logging.INFO)

# Create a rotating file handler
handler = RotatingFileHandler('aviator_game.log', maxBytes=1000000, backupCount=1)
handler.setLevel(logging.INFO)

# Create a logging format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)

def discover_urls(base_url):
    """
    Discover data and bet URLs automatically from the base URL.
    """
    try:
        response = requests.get(base_url)
        response.raise_for_status()
        data = response.json()
        data_url = data.get('data_url')
        bet_url = data.get('bet_url')
        return data_url, bet_url
    except requests.exceptions.RequestException as e:
        logger.error(f'Error discovering URLs: {e}')
        return None, None

def collect_data(data_url, rounds):
    """
    Collect game data for a specified number of rounds.
    """
    try:
        response = requests.get(data_url, params={'rounds': rounds})
        response.raise_for_status()
        data = response.json()
        df = pd.DataFrame(data)
        return df
    except requests.exceptions.RequestException as e:
        logger.error(f'Error collecting data: {e}')
        return None

def train_model(df):
    """
    Train a Random Forest model with hyperparameter tuning.
    """
    try:
        df.dropna(inplace=True)

        features = ['crash_point_lag_1', 'crash_point_lag_2', 'crash_point_lag_3', 'crash_point_lag_4']
        target = 'crash_point'

        X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

        # Hyperparameter tuning using Grid Search
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [None, 5, 10],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 5, 10]
        }

        model = RandomForestRegressor(random_state=42)
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring=make_scorer(mean_squared_error, greater_is_better=False))
        grid_search.fit(X_train, y_train)

        best_model = grid_search.best_estimator_
        best_model.fit(X_train, y_train)

        return best_model
    except Exception as e:
        logger.error(f'Error training model: {e}')
        return None

def evaluate_model(model, X_test, y_test):
    """
    Evaluate the trained model using Mean Absolute Error.
    """
    try:
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        return mae
    except Exception as e:
        logger.error(f'Error evaluating model: {e}')
        return None

def predict_next_crash_point(model, lagged_points):
    """
    Predict the next crash point based on lagged data.
    """
    try:
        return model.predict([lagged_points])[0]
    except Exception as e:
        logger.error(f'Error predicting next crash point: {e}')
        return None

def should_place_bet(predicted_crash_point, threshold):
    """
    Decide whether to place a bet based on the predicted crash point.
    """
    try:
        return predicted_crash_point >= threshold
    except Exception as e:
        logger.error(f'Error determining whether to place bet: {e}')
        return False

def place_bet(amount, predicted_crash_point, bet_url, auth_token):
    """
    Place a bet using the Aviator game API.
    """
    try:
        headers = {
            'Authorization': f'Bearer {auth_token}',
            'Content-Type': 'application/json'
        }
        bet_data = {
            'amount': amount,
            'predicted_crash_point': predicted_crash_point
        }

        response = requests.post(bet_url, headers=headers, data=json.dumps(bet_data))
        response.raise_for_status()
        logger.info(f'Bet placed successfully: {response.json()}')
    except requests.exceptions.RequestException as e:
        logger.error(f'Error placing bet: {e}')

def visualize_data(df):
    """
    Visualize the crash point trend.
    """
    try:
        plt.figure(figsize=(10, 6))
        plt.plot(df['crash_point'], label='Crash Point')
        plt.axhline(y=THRESHOLD, color='r', linestyle='--', label='Bet Threshold')
        plt.title('Crash Point Trend')
        plt.xlabel('Round')
        plt.ylabel('Crash Point')
        plt.legend()
        plt.show()
    except Exception as e:
        logger.error(f'Error visualizing data: {e}')

def main():
    # Discover URLs
    data_url, bet_url = discover_urls(BASE_URL)
    if data_url is None or bet_url is None:
        logger.error('Failed to discover URLs. Exiting.')
        return

    # Collect data
    df = collect_data(data_url, ROUNDS)
    if df is None:
        logger.error('Failed to collect data. Exiting.')
        return

    # Train model
    model = train_model(df)
    if model is None:
        logger.error('Failed to train model. Exiting.')
        return

    # Prepare data for evaluation
    features = ['crash_point_lag_1', 'crash_point_lag_2', 'crash_point_lag_3', 'crash_point_lag_4']
    X_test = df[features].iloc[-20:]  # Last 20 rounds for testing
    y_test = df['crash_point'].iloc[-20:]

    # Evaluate model
    mae = evaluate_model(model, X_test, y_test)
    if mae is not None:
        logger.info(f'Model Evaluation - Mean Absolute Error: {mae}')

    # Predict next crash point
    last_lagged_points = df[features].iloc[-1].values
    predicted_crash_point = predict_next_crash_point(model, last_lagged_points)
    logger.info(f'Predicted Next Crash Point: {predicted_crash_point}')

    # Decide whether to place a bet
    if should_place_bet(predicted_crash_point, THRESHOLD):
        place_bet(BET_AMOUNT, predicted_crash_point, bet_url, AUTH_TOKEN)
    else:
        logger.info('No bet placed. Predicted crash point below threshold.')

    # Visualize data
    visualize_data(df)

if __name__ == '__main__':
    main()


ImportError: cannot import name 'requests' from 'google.colab' (/usr/local/lib/python3.10/dist-packages/google/colab/__init__.py)